In [5]:
import pandas as pd
import numpy as np
import itertools
import random
from tqdm import tqdm

pd.set_option('display.max_columns', 50)

In [6]:
reg_data_m =pd.read_csv('data/MRegularSeasonDetailedResults.csv')
reg_data_m.head()

,Season,DayNum,WTeamID,WScore,LTeamID,LScore,WLoc,NumOT,WFGM,WFGA,WFGM3,WFGA3,WFTM,WFTA,WOR,WDR,WAst,WTO,WStl,WBlk,WPF,LFGM,LFGA,LFGM3,LFGA3,LFTM,LFTA,LOR,LDR,LAst,LTO,LStl,LBlk,LPF
0,2003,10,1104,68,1328,62,N,0,27,58,3,14,11,18,14,24,13,23,7,1,22,22,53,2,10,16,22,10,22,8,18,9,2,20
1,2003,10,1272,70,1393,63,N,0,26,62,8,20,10,19,15,28,16,13,4,4,18,24,67,6,24,9,20,20,25,7,12,8,6,16
2,2003,11,1266,73,1437,61,N,0,24,58,8,18,17,29,17,26,15,10,5,2,25,22,73,3,26,14,23,31,22,9,12,2,5,23
3,2003,11,1296,56,1457,50,N,0,18,38,3,9,17,31,6,19,11,12,14,2,18,18,49,6,22,8,15,17,20,9,19,4,3,23
4,2003,11,1400,77,1208,71,N,0,30,61,6,14,11,13,17,22,12,14,4,4,20,24,62,6,16,17,27,21,15,12,10,7,1,14


In [7]:
ncaa_data_m = pd.read_csv('data/MNCAATourneyDetailedResults.csv')
ncaa_data_m.head()

,Season,DayNum,WTeamID,WScore,LTeamID,LScore,WLoc,NumOT,WFGM,WFGA,WFGM3,WFGA3,WFTM,WFTA,WOR,WDR,WAst,WTO,WStl,WBlk,WPF,LFGM,LFGA,LFGM3,LFGA3,LFTM,LFTA,LOR,LDR,LAst,LTO,LStl,LBlk,LPF
0,2003,134,1421,92,1411,84,N,1,32,69,11,29,17,26,14,30,17,12,5,3,22,29,67,12,31,14,31,17,28,16,15,5,0,22
1,2003,136,1112,80,1436,51,N,0,31,66,7,23,11,14,11,36,22,16,10,7,8,20,64,4,16,7,7,8,26,12,17,10,3,15
2,2003,136,1113,84,1272,71,N,0,31,59,6,14,16,22,10,27,18,9,7,4,19,25,69,7,28,14,21,20,22,11,12,2,5,18
3,2003,136,1141,79,1166,73,N,0,29,53,3,7,18,25,11,20,15,18,13,1,19,27,60,7,17,12,17,14,17,20,21,6,6,21
4,2003,136,1143,76,1301,74,N,1,27,64,7,20,15,23,18,20,17,13,8,2,14,25,56,9,21,15,20,10,26,16,14,5,8,19


In [8]:
#removing WLoc
print(reg_data_m['WLoc'].value_counts())
print(ncaa_data_m['WLoc'].value_counts())
# removing WLoc because the final data to predicted on will only have it as Neutral so not needed

H    63049
A    33019
N    10766
Name: WLoc, dtype: int64
N    1248
Name: WLoc, dtype: int64


In [9]:
# thinking of removing NumOT
print("Regular Season\n",reg_data_m['NumOT'].value_counts())
print("NCAA\n",ncaa_data_m['NumOT'].value_counts())
# since NumOT is generally 0 and it is 1 for very few samples in the NCAA data, i believe it to not contribute a whole lot

Regular Season
 0    100790
1      5004
2       844
3       157
4        33
5         5
6         1
Name: NumOT, dtype: int64
NCAA
 0    1170
1      65
2      13
Name: NumOT, dtype: int64


In [10]:
reg_data_w =pd.read_csv('data/WRegularSeasonDetailedResults.csv')
reg_data_w.head()

,Season,DayNum,WTeamID,WScore,LTeamID,LScore,WLoc,NumOT,WFGM,WFGA,WFGM3,WFGA3,WFTM,WFTA,WOR,WDR,WAst,WTO,WStl,WBlk,WPF,LFGM,LFGA,LFGM3,LFGA3,LFTM,LFTA,LOR,LDR,LAst,LTO,LStl,LBlk,LPF
0,2010,11,3103,63,3237,49,H,0,23,54,5,9,12,19,10,26,14,18,7,0,15,20,54,3,13,6,10,11,27,11,23,7,6,19
1,2010,11,3104,73,3399,68,N,0,26,62,5,12,16,28,16,31,15,20,5,2,25,25,63,4,21,14,27,14,26,7,20,4,2,27
2,2010,11,3110,71,3224,59,A,0,29,62,6,15,7,12,14,23,18,13,6,2,17,19,58,2,14,19,23,17,23,8,15,6,0,15
3,2010,11,3111,63,3267,58,A,0,27,52,4,11,5,9,6,40,14,27,5,10,18,18,74,6,26,16,25,22,22,15,11,14,5,14
4,2010,11,3119,74,3447,70,H,1,30,74,7,20,7,11,14,33,18,11,5,3,18,25,74,9,17,11,21,21,32,12,14,4,2,14


In [11]:
ncaa_data_w = pd.read_csv('data/WNCAATourneyDetailedResults.csv')
ncaa_data_w.head()

,Season,DayNum,WTeamID,WScore,LTeamID,LScore,WLoc,NumOT,WFGM,WFGA,WFGM3,WFGA3,WFTM,WFTA,WOR,WDR,WAst,WTO,WStl,WBlk,WPF,LFGM,LFGA,LFGM3,LFGA3,LFTM,LFTA,LOR,LDR,LAst,LTO,LStl,LBlk,LPF
0,2010,138,3124,69,3201,55,N,0,28,57,1,5,12,19,13,24,22,12,6,2,12,21,61,10,34,3,5,17,19,12,18,4,1,18
1,2010,138,3173,67,3395,66,N,0,23,59,9,26,12,19,13,34,13,16,3,10,14,22,73,8,27,14,15,18,26,8,8,8,6,22
2,2010,138,3181,72,3214,37,H,0,26,57,4,13,16,22,13,34,15,11,10,7,11,15,56,4,15,3,8,10,21,4,16,6,4,20
3,2010,138,3199,75,3256,61,H,0,25,63,3,15,22,26,20,27,13,17,8,3,21,21,62,2,20,17,22,16,21,13,16,5,4,24
4,2010,138,3207,62,3265,42,N,0,24,68,8,25,6,8,20,29,16,8,5,5,18,13,60,5,26,11,17,16,22,9,10,3,4,12


In [12]:
#removing WLoc
print(reg_data_w['WLoc'].value_counts())
print(ncaa_data_w['WLoc'].value_counts())
# removing WLoc because the final data to predicted on will only have it as Neutral so not needed

H    38742
A    25171
N     6094
Name: WLoc, dtype: int64
N    492
H    228
A     40
Name: WLoc, dtype: int64


In [13]:
# thinking of removing NumOT
print("Regular Season\n",reg_data_w['NumOT'].value_counts())
print("NCAA\n",ncaa_data_w['NumOT'].value_counts())
# since NumOT is generally 0 and it is 1 for very few samples in the NCAA data, i believe it to not contribute a whole lot

Regular Season
 0    66911
1     2633
2      397
3       54
4       11
5        1
Name: NumOT, dtype: int64
NCAA
 0    734
1     23
2      3
Name: NumOT, dtype: int64


In [14]:
data = pd.concat([reg_data_m,ncaa_data_m,reg_data_w,ncaa_data_w])
print(data.shape)
print(data['Season'].value_counts())

(178849, 34)
2019    10833
2018    10744
2017    10735
2014    10718
2016    10708
2015    10698
2013    10593
2022    10539
2020    10499
2011    10460
2012    10433
2010    10427
2023     9400
2021     7540
2009     5313
2008     5227
2007     5107
2006     4821
2005     4739
2003     4680
2004     4635
Name: Season, dtype: int64


In [17]:
drop_cols = ['WLoc','NumOT']
reg_data = data.drop(columns = drop_cols)
cols = reg_data.columns.values
print(cols)
reg_data.head()

['Season' 'DayNum' 'WTeamID' 'WScore' 'LTeamID' 'LScore' 'WFGM' 'WFGA'
 'WFGM3' 'WFGA3' 'WFTM' 'WFTA' 'WOR' 'WDR' 'WAst' 'WTO' 'WStl' 'WBlk'
 'WPF' 'LFGM' 'LFGA' 'LFGM3' 'LFGA3' 'LFTM' 'LFTA' 'LOR' 'LDR' 'LAst'
 'LTO' 'LStl' 'LBlk' 'LPF']


,Season,DayNum,WTeamID,WScore,LTeamID,LScore,WFGM,WFGA,WFGM3,WFGA3,WFTM,WFTA,WOR,WDR,WAst,WTO,WStl,WBlk,WPF,LFGM,LFGA,LFGM3,LFGA3,LFTM,LFTA,LOR,LDR,LAst,LTO,LStl,LBlk,LPF
0,2003,10,1104,68,1328,62,27,58,3,14,11,18,14,24,13,23,7,1,22,22,53,2,10,16,22,10,22,8,18,9,2,20
1,2003,10,1272,70,1393,63,26,62,8,20,10,19,15,28,16,13,4,4,18,24,67,6,24,9,20,20,25,7,12,8,6,16
2,2003,11,1266,73,1437,61,24,58,8,18,17,29,17,26,15,10,5,2,25,22,73,3,26,14,23,31,22,9,12,2,5,23
3,2003,11,1296,56,1457,50,18,38,3,9,17,31,6,19,11,12,14,2,18,18,49,6,22,8,15,17,20,9,19,4,3,23
4,2003,11,1400,77,1208,71,30,61,6,14,11,13,17,22,12,14,4,4,20,24,62,6,16,17,27,21,15,12,10,7,1,14


In [18]:
# converting it to team A and team B from WTeam and LTeam
list_reg_data = []

for row in tqdm(reg_data.itertuples()):
    new_reg_data = dict()
    coin = random.random()
    new_reg_data['Season'] = row.Season
    new_reg_data['DayNum'] = row.DayNum

    if coin>0.5:
        new_reg_data['A'] = row.WTeamID
        new_reg_data['B'] = row.LTeamID
        try:
            new_reg_data['FGP_A'] = (row.WFGM / row.WFGA) * 100
        except:
            new_reg_data['FGP_A'] = 0
        try:
            new_reg_data['FGP3_A'] = (row.WFGM3 / row.WFGA3) * 100
        except:
            new_reg_data['FGP3_A'] = 0
        try:
            new_reg_data['FTP_A'] = (row.WFTM / row.WFTA) * 100
        except:
            new_reg_data['FTP_A'] = 0
        new_reg_data['OR_A'] = row.WOR
        new_reg_data['DR_A'] = row.WDR
        new_reg_data['AST_A'] = row.WAst
        new_reg_data['TO_A'] = row.WTO
        new_reg_data['STL_A'] = row.WStl
        new_reg_data['BLK_A'] = row.WBlk
        new_reg_data['PF_A'] = row.WPF
        new_reg_data['PM_A'] = row.WScore - row.LScore # plus minus for team A
        new_reg_data['POSS_A'] = row.WFGA - row.
        try:
            new_reg_data['FGP_B'] = (row.LFGM / row.LFGA) * 100
        except:
            new_reg_data['FGP_B'] = 0
        try:
            new_reg_data['FGP3_B'] = (row.LFGM3 / row.LFGA3) * 100
        except:
            new_reg_data['FGP3_B'] = 0
        try:
            new_reg_data['FTP_B'] = (row.LFTM / row.LFTA) * 100
        except:
            new_reg_data['FTP_B'] = 0
        new_reg_data['OR_B'] = row.LOR
        new_reg_data['DR_B'] = row.LDR
        new_reg_data['AST_B'] = row.LAst
        new_reg_data['TO_B'] = row.LTO
        new_reg_data['STL_B'] = row.LStl
        new_reg_data['BLK_B'] = row.LBlk
        new_reg_data['PF_B'] = row.LPF
        new_reg_data['PM_B'] = row.LScore - row.WScore # plus minus for team B
        new_reg_data['Win'] = 1
    else:
        new_reg_data['A'] = row.LTeamID
        new_reg_data['B'] = row.WTeamID
        try:
            new_reg_data['FGP_A'] = (row.LFGM / row.LFGA) * 100
        except:
            new_reg_data['FGP_A'] = 0
        try:
            new_reg_data['FGP3_A'] = (row.LFGM3 / row.LFGA3) * 100
        except:
            new_reg_data['FGP3_A'] = 0
        try:
            new_reg_data['FTP_A'] = (row.LFTM / row.LFTA) * 100
        except:
            new_reg_data['FTP_A'] = 0
        new_reg_data['OR_A'] = row.LOR
        new_reg_data['DR_A'] = row.LDR
        new_reg_data['AST_A'] = row.LAst
        new_reg_data['TO_A'] = row.LTO
        new_reg_data['STL_A'] = row.LStl
        new_reg_data['BLK_A'] = row.LBlk
        new_reg_data['PF_A'] = row.LPF
        new_reg_data['PM_A'] = row.LScore - row.WScore # plus minus for team A
        try:
            new_reg_data['FGP_B'] = (row.WFGM / row.WFGA) * 100
        except:
            new_reg_data['FGP_B'] = 0
        try:
            new_reg_data['FGP3_B'] = (row.WFGM3 / row.WFGA3) * 100
        except:
            new_reg_data['FGP3_B'] = 0
        try:
            new_reg_data['FTP_B'] = (row.WFTM / row.WFTA) * 100
        except:
            new_reg_data['FTP_B'] = 0
        new_reg_data['OR_B'] = row.WOR
        new_reg_data['DR_B'] = row.WDR
        new_reg_data['AST_B'] = row.WAst
        new_reg_data['TO_B'] = row.WTO
        new_reg_data['STL_B'] = row.WStl
        new_reg_data['BLK_B'] = row.WBlk
        new_reg_data['PF_B'] = row.WPF
        new_reg_data['PM_B'] = row.WScore - row.LScore # plus minus for team B
        new_reg_data['Win'] = 0

    list_reg_data.append(new_reg_data)

new_reg_data = pd.DataFrame(list_reg_data)
new_reg_data


178849it [00:00, 242668.03it/s]


,Season,DayNum,A,B,FGP_A,FGP3_A,FTP_A,OR_A,DR_A,AST_A,TO_A,STL_A,BLK_A,PF_A,PM_A,FGP_B,FGP3_B,FTP_B,OR_B,DR_B,AST_B,TO_B,STL_B,BLK_B,PF_B,PM_B,Win
0,2003,10,1104,1328,46.551724,21.428571,61.111111,14,24,13,23,7,1,22,6,41.509434,20.000000,72.727273,10,22,8,18,9,2,20,-6,1
1,2003,10,1393,1272,35.820896,25.000000,45.000000,20,25,7,12,8,6,16,-7,41.935484,40.000000,52.631579,15,28,16,13,4,4,18,7,0
2,2003,11,1266,1437,41.379310,44.444444,58.620690,17,26,15,10,5,2,25,12,30.136986,11.538462,60.869565,31,22,9,12,2,5,23,-12,1
3,2003,11,1457,1296,36.734694,27.272727,53.333333,17,20,9,19,4,3,23,-6,47.368421,33.333333,54.838710,6,19,11,12,14,2,18,6,0
4,2003,11,1208,1400,38.709677,37.500000,62.962963,21,15,12,10,7,1,14,-6,49.180328,42.857143,84.615385,17,22,12,14,4,4,20,6,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
178844,2022,147,3301,3163,48.484848,30.434783,84.210526,6,30,20,13,4,7,16,-4,48.051948,23.809524,60.000000,12,23,10,7,5,2,16,4,0
178845,2022,147,3276,3257,34.782609,21.428571,75.000000,10,24,9,21,6,2,12,-12,43.103448,33.333333,77.777778,6,20,12,11,15,4,17,12,0
178846,2022,151,3390,3163,34.848485,17.391304,61.538462,11,23,10,11,11,3,16,-5,36.842105,35.714286,80.000000,12,30,14,19,5,2,16,5,0
178847,2022,151,3257,3376,42.857143,12.500000,57.142857,11,18,5,15,13,2,17,-13,47.368421,35.294118,70.588235,8,24,19,14,11,4,11,13,0


In [19]:
new_reg_data['Win'].value_counts()

0    89481
1    89368
Name: Win, dtype: int64

In [20]:
new_reg_data.corr()

,Season,DayNum,A,B,FGP_A,FGP3_A,FTP_A,OR_A,DR_A,AST_A,TO_A,STL_A,BLK_A,PF_A,PM_A,FGP_B,FGP3_B,FTP_B,OR_B,DR_B,AST_B,TO_B,STL_B,BLK_B,PF_B,PM_B,Win
Season,1.000000,-0.040249,0.290901,0.290655,-0.052835,-0.052611,0.051929,-0.166480,0.068099,-0.018628,-0.073808,-0.017853,-0.009880,-0.167825,0.000640,-0.055034,-0.054418,0.055065,-0.166198,0.065799,-0.019730,-0.077026,-0.013990,-0.012072,-0.166476,-0.000640,0.000944
DayNum,-0.040249,1.000000,-0.013725,-0.013848,0.026498,0.021329,0.061273,-0.060600,-0.037456,-0.025925,-0.148152,-0.069690,-0.018510,-0.049894,0.003451,0.024410,0.023748,0.062444,-0.062237,-0.037920,-0.031510,-0.145027,-0.072886,-0.020609,-0.050639,-0.003451,0.002920
A,0.290901,-0.013725,1.000000,0.988723,-0.238376,-0.126763,-0.008010,0.161683,0.106228,-0.006545,0.283056,0.205573,-0.006946,-0.083000,0.004039,-0.247234,-0.132232,-0.005990,0.165951,0.096254,-0.014913,0.279085,0.208134,-0.011993,-0.079698,-0.004039,0.001346
B,0.290655,-0.013848,0.988723,1.000000,-0.240429,-0.126992,-0.008656,0.160765,0.104605,-0.007516,0.283625,0.205608,-0.006811,-0.081017,0.001109,-0.245316,-0.132075,-0.005812,0.166950,0.098241,-0.013577,0.278886,0.208234,-0.011947,-0.081099,-0.001109,-0.001471
FGP_A,-0.052835,0.026498,-0.238376,-0.240429,1.000000,0.528587,0.075896,-0.310439,0.141875,0.565653,-0.090455,-0.003572,0.097411,-0.051068,0.582498,-0.051540,-0.000313,0.007035,-0.071362,-0.647587,-0.119857,-0.079396,-0.089141,-0.321922,0.080420,-0.582498,0.480145
FGP3_A,-0.052611,0.021329,-0.126763,-0.126992,0.528587,1.000000,0.059836,-0.207580,0.049737,0.413239,-0.062346,-0.051862,0.026969,-0.027785,0.355257,-0.002463,0.009845,0.011694,-0.012448,-0.345112,-0.040056,-0.086488,-0.058687,-0.082795,0.009249,-0.355257,0.303068
FTP_A,0.051929,0.061273,-0.008010,-0.008656,0.075896,0.059836,1.000000,-0.111954,0.020520,0.041996,-0.064095,-0.025681,-0.002618,-0.034150,0.156658,0.005387,0.011928,0.038627,-0.035934,-0.212634,-0.013994,-0.047773,-0.044335,-0.039041,0.017715,-0.156658,0.137809
OR_A,-0.166480,-0.060600,0.161683,0.160765,-0.310439,-0.207580,-0.111954,1.000000,0.084158,0.016276,0.117193,0.165883,0.079849,0.100410,0.079552,-0.063494,-0.013264,-0.036549,0.089801,0.026258,-0.009569,0.198248,0.075286,0.232973,0.144945,-0.079552,0.019788
DR_A,0.068099,-0.037456,0.106228,0.104605,0.141875,0.049737,0.020520,0.084158,1.000000,0.219728,0.130482,-0.096369,0.262401,-0.038205,0.452517,-0.648328,-0.339925,-0.205955,0.026577,0.016238,-0.267983,-0.169241,0.082536,-0.055385,0.138769,-0.452517,0.385764
AST_A,-0.018628,-0.025925,-0.006545,-0.007516,0.565653,0.413239,0.041996,0.016276,0.219728,1.000000,-0.099357,0.138338,0.149799,-0.072756,0.522538,-0.118477,-0.039750,-0.011590,-0.013349,-0.267825,-0.017617,0.094679,-0.047847,-0.146123,-0.008515,-0.522538,0.393369


### Few heavy correlation
- FGP_A - FGP3_A (0.53)
- FGP_A - AST_A (0.55)
- FGP_A - DR_B (negative)
- FGP3_A - AST_A
- TO_A - STL_B (quite high, 0.71)
- STL_A - TO_B (quite high, 0.71)
same for B

### Features
- Win %? O.o
- Plus_Minus

In [11]:
drop_cols = ['STL_A','STL_B']
new_reg_data = new_reg_data.drop(columns = drop_cols)
cols = new_reg_data.columns.values
print(cols)
new_reg_data.head()

['Season' 'DayNum' 'NumOT' 'A' 'B' 'FGP_A' 'FGP3_A' 'FTP_A' 'OR_A' 'DR_A'
 'AST_A' 'TO_A' 'BLK_A' 'PF_A' 'FGP_B' 'FGP3_B' 'FTP_B' 'OR_B' 'DR_B'
 'AST_B' 'TO_B' 'BLK_B' 'PF_B' 'Win']


,Season,DayNum,NumOT,A,B,FGP_A,FGP3_A,FTP_A,OR_A,DR_A,AST_A,TO_A,BLK_A,PF_A,FGP_B,FGP3_B,FTP_B,OR_B,DR_B,AST_B,TO_B,BLK_B,PF_B,Win
0,2003,10,0,1328,1104,41.509434,20.000000,72.727273,10,22,8,18,2,20,46.551724,21.428571,61.111111,14,24,13,23,1,22,0
1,2003,10,0,1393,1272,35.820896,25.000000,45.000000,20,25,7,12,6,16,41.935484,40.000000,52.631579,15,28,16,13,4,18,0
2,2003,11,0,1437,1266,30.136986,11.538462,60.869565,31,22,9,12,5,23,41.379310,44.444444,58.620690,17,26,15,10,2,25,0
3,2003,11,0,1296,1457,47.368421,33.333333,54.838710,6,19,11,12,2,18,36.734694,27.272727,53.333333,17,20,9,19,3,23,1
4,2003,11,0,1400,1208,49.180328,42.857143,84.615385,17,22,12,14,4,20,38.709677,37.500000,62.962963,21,15,12,10,1,14,1


In [12]:
def get_avg_team(data, team_id, dn):
    data = data[(data['A'] == team_id)|(data['B'] == team_id)]
    data = data[data['DayNum']<dn]
    final_data = {'FGP':[],'FGP3':[],'FTP':[],'OR':[],'DR':[],'AST':[],'TO':[],'BLK':[],'PF':[]}
    for row in (data.itertuples()):
        if row.A == team_id:
            final_data['FGP'].append(row.FGP_A)
            final_data['FGP3'].append(row.FGP3_A)
            final_data['FTP'].append(row.FTP_A)
            final_data['OR'].append(row.OR_A)
            final_data['DR'].append(row.DR_A)
            final_data['AST'].append(row.AST_A)
            final_data['TO'].append(row.TO_A)
            final_data['BLK'].append(row.BLK_A)
            final_data['PF'].append(row.PF_A)
        else:
            final_data['FGP'].append(row.FGP_B)
            final_data['FGP3'].append(row.FGP3_B)
            final_data['FTP'].append(row.FTP_B)
            final_data['OR'].append(row.OR_B)
            final_data['DR'].append(row.DR_B)
            final_data['AST'].append(row.AST_B)
            final_data['TO'].append(row.TO_B)
            final_data['BLK'].append(row.BLK_B)
            final_data['PF'].append(row.PF_B)    
    try:
        return [sum(v)/len(v) for v in final_data.values()]
    except:
        return [np.nan]*len(final_data)


def get_avg_yearly(year):
    year_data = new_reg_data[new_reg_data['Season'] == year]
    list_data = []

    for row in tqdm(year_data.itertuples()):
        *stats_A, = get_avg_team(year_data, row.A, row.DayNum)
        *stats_B, = get_avg_team(year_data, row.B, row.DayNum)
        list_data.append([row.Season, row.DayNum, row.A, row.B, *stats_A, *stats_B, row.Win])
    
    data = pd.DataFrame(list_data, columns = [['Season','DayNum','A','B','FGP_A','FGP3_A','FTP_A','OR_A','DR_A',
                                               'AST_A','TO_A','BLK_A','PF_A','FGP_B','FGP3_B','FTP_B','OR_B','DR_B',
                                               'AST_B','TO_B','BLK_B','PF_B','Win']])
    
    data = data.dropna()

    return data

def get_data_reg():
    year = [2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020, 2021, 2022, 2023]
    data = []
    for y in tqdm(year):
        data.append(get_avg_yearly(y))

    final_data = pd.concat(data)
    return final_data


In [13]:
final_data = get_data_reg()

4616it [00:11, 417.69it/s]00<?, ?it/s]
4571it [00:10, 427.02it/s]11<03:41, 11.07s/it]
4675it [00:10, 429.07it/s]21<03:26, 10.86s/it]
4757it [00:11, 428.84it/s]32<03:15, 10.88s/it]
5043it [00:11, 428.47it/s]43<03:06, 10.97s/it]
5163it [00:12, 428.58it/s]55<03:00, 11.26s/it]
5249it [00:12, 429.81it/s]07<02:53, 11.54s/it]
5263it [00:12, 429.52it/s]19<02:44, 11.76s/it]
5246it [00:12, 430.14it/s]32<02:34, 11.92s/it]
5253it [00:12, 428.20it/s]44<02:24, 12.01s/it]
5320it [00:12, 427.94it/s]:56<02:13, 12.10s/it]
5362it [00:12, 429.30it/s]:09<02:02, 12.20s/it]
5354it [00:12, 428.65it/s]:21<01:50, 12.30s/it]
5369it [00:12, 426.78it/s]:34<01:38, 12.36s/it]
5395it [00:12, 429.33it/s]:46<01:27, 12.43s/it]
5405it [00:12, 427.83it/s]:59<01:14, 12.48s/it]
5463it [00:12, 428.30it/s]:11<01:02, 12.53s/it]
5328it [00:12, 423.83it/s]:24<00:50, 12.60s/it]
3855it [00:08, 430.76it/s]:37<00:37, 12.60s/it]
5345it [00:12, 430.04it/s]:46<00:23, 11.51s/it]
4802it [00:12, 392.39it/s]:58<00:11, 11.79s/it]
100%|█████

In [14]:
final_data.to_csv('computed_data/reg_avg_data.csv',index=False)

In [3]:
final_data = pd.read_csv('computed_data/reg_avg_data.csv')

In [4]:
final_data.shape

(102373, 24)